In [190]:
import pandas as pd
import ast
from sklearn.preprocessing import MultiLabelBinarizer


In [191]:
df=pd.read_csv('ModApte_test.csv')

In [192]:
def series_to_list(df,column_name):
    result=df[column_name].replace({' ':''},regex=True)
    result.replace({'\\n':''},regex=True,inplace=True)
    result.replace({'\'\'':'\',\''},regex=True,inplace=True)
    return result.apply(ast.literal_eval)

In [193]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3299 entries, 0 to 3298
Data columns (total 13 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   text         3023 non-null   object
 1   text_type    3299 non-null   object
 2   topics       3299 non-null   object
 3   lewis_split  3299 non-null   object
 4   cgis_split   3299 non-null   object
 5   old_id       3299 non-null   object
 6   new_id       3299 non-null   object
 7   places       3299 non-null   object
 8   people       3299 non-null   object
 9   orgs         3299 non-null   object
 10  exchanges    3299 non-null   object
 11  date         3299 non-null   object
 12  title        3285 non-null   object
dtypes: object(13)
memory usage: 335.2+ KB


In [194]:
df.dtypes

text           object
text_type      object
topics         object
lewis_split    object
cgis_split     object
old_id         object
new_id         object
places         object
people         object
orgs           object
exchanges      object
date           object
title          object
dtype: object

In [195]:
df.head()

text text_type  \
0  Mounting trade friction between the\nU.S. And ...    "NORM"   
1  A survey of 19 provinces and seven cities\nsho...    "NORM"   
2  The Ministry of International Trade and\nIndus...    "NORM"   
3  Thailand's trade deficit widened to 4.5\nbilli...    "NORM"   
4  Indonesia expects crude palm oil (CPO)\nprices...    "NORM"   

                                              topics lewis_split  \
0                                          ['trade']      "TEST"   
1                                          ['grain']      "TEST"   
2                                ['crude' 'nat-gas']      "TEST"   
3  ['trade' 'grain' 'rice' 'corn' 'sugar' 'tin' '...      "TEST"   
4                             ['veg-oil' 'palm-oil']      "TEST"   

       cgis_split  old_id   new_id  \
0  "TRAINING-SET"  "3809"  "14826"   
1  "TRAINING-SET"  "3811"  "14828"   
2  "TRAINING-SET"  "4356"  "14829"   
3  "TRAINING-SET"  "3815"  "14832"   
4  "TRAINING-SET"  "3816"  "14833"   

                                              places people orgs exchanges  \
0  ['hong-kong' 'usa' 'japan' 'taiwan' 'malaysia'...     []   []        []   
1                                          ['china']     []   []        []   
2                                          ['japan']     []   []        []   
3                                       ['thailand']     []   []        []   
4                           ['indonesia' 'malaysia']     []   []        []   

                      date                                              title  
0   8-APR-1987 01:03:47.52   ASIAN EXPORTERS FEAR DAMAGE FROM U.S.-JAPAN RIFT  
1   8-APR-1987 01:19:17.29  CHINA DAILY SAYS VERMIN EAT 7-12 PCT GRAIN STOCKS  
2   8-APR-1987 01:22:17.25  JAPAN TO REVISE LONG-TERM ENERGY DEMAND DOWNWARDS  
3   8-APR-1987 01:45:09.09         THAI TRADE DEFICIT WIDENS IN FIRST QUARTER  
4   8-APR-1987 01:48:20.11            INDONESIA SEES CPO PRICE RISING SHARPLY

In [196]:
#define topics
topic_column = 'topics'
food = ['coconut', 'cotton-oil', 'sorghum', 'orange', 'rice', 'soybean', 'sun-meal', 
    'oilseed', 'sugar', 'hog', 'coffee', 'groundnut', 'sunseed', 'sun-oil', 'rye', 
    'lin-oil', 'copra-cake', 'potato', 'barley', 'tea', 'meal-feed', 'coconut-oil', 
    'palmkernel', 'cottonseed', 'castor-oil', 'l-cattle', 'livestock', 'soy-oil', 
    'rape-oil', 'palm-oil', 'cocoa', 'cotton', 'wheat', 'corn', 'f-cattle', 'grain', 
    'soy-meal', 'oat', 'groundnut-oil', 'veg-oil','rapeseed']
resource = ['platinum', 'lead', 'nickel', 'strategic-metal', 'copper', 'palladium', 'gold', 
    'zinc', 'tin', 'iron-steel', 'alum', 'silver', 'nat-gas', 'rubber', 'pet-chem', 'fuel', 'crude','lumber','propane']
finance = ['money-supply', 'dlr', 'nkr', 'lei', 'yen', 'dfl', 'sfr', 'cpi', 'instal-debt', 
    'money-fx', 'gnp', 'interest', 'income', 'dmk', 'rand', 'bop', 'reserves', 'nzdlr','acq']
personal_finance = ['housing','jobs','earn']
transport = ['jet', 'ship']
topics=[[food,'food'],[resource,'resource'],[finance,'finance'],[personal_finance,'personal_finance'],[transport,'transport']]
topics_to_remove = ['gas', 'heat', 'trade', 'retail', 'carcass', 'cpu', 'wpi', 'naphtha', 'ipi']

#columns with special treatment
list_columns=['places']
drop_columns=['people','orgs','exchanges','lewis_split','cgis_split','old_id','new_id']
notnan_columns=['text','topics']
date_columns=[['date',True]]#[Date,Feature_Extraction]
text_columns=['text','title']

In [197]:
def drop_row_notnan_columms(df,notnan_columns):
    for column in notnan_columns:
        df[column].dropna(inplace=True)
    return df


In [198]:
#funtion to reorganize a column of subtopics into  broader topics and removing some of them 
def categorize_topics(df,column,topics,remove):
    for topic in topics:
        for subtopic in topic[0]:
            df[column]=df[column].replace({'\''+subtopic+'\'':'\''+topic[1] +'\''},regex=True)
    for subtopic in remove:
        df[column]=df[column].replace({'\''+subtopic+'\'':''},regex=True)
    df[column]=df[column].replace({' ':''},regex=True)
    df[column]=series_to_list(df,column)
    df=df[df[column].str.len()==1]
    df[column]=df[column].apply(lambda x : x[0])
    return df

In [199]:
def explode_listcolumns(df,list_columns):
    for column in list_columns:
        df[column]=series_to_list(df,column)
        df = df.join(pd.crosstab((s:=df[column].explode()).index, s).add_prefix(column+'_'))
        df = df.drop(columns=column)
        for col  in [col for col in df if col.startswith(column+'_')]:
            df[col].fillna(value=0,inplace=True)
    return df

In [200]:
def format_datecolumns(df,date_columns):
    for column in date_columns:
        # Die Zeichenkette in ein Datum konvertieren
        df[column[0]] = pd.to_datetime(df[column[0]])
        if column[1]:
            # Monat extrahieren
            df[column[0]+'_month'] = df[column[0]].dt.month

            # Woche extrahieren (altes Verhalten, ab Pandas 1.1.0 ist isocalendar().week empfohlen)
            df[column[0]+'_day_month'] = df[column[0]].dt.day

            # Tag extrahieren
            df[column[0]+'_day_year'] = df[column[0]].dt.day_of_year

            # Wochentag extrahieren (Montag=0, Sonntag=6)
            df[column[0]+'_weekday'] = df[column[0]].dt.day_name('en')

            df[column[0]+'_quarter_year'] = df[column[0]].dt.quarter
            df=pd.get_dummies(df, columns=[column[0]+'_weekday'])
            weekdays=['weekday_Monday','weekday_Tuesday','weekday_Wednesday','weekday_Thursday','weekday_Friday','weekday_Saturday','weekday_Sunday']
            for weekday in weekdays:
                if not weekday in df.columns:
                    df[weekday]=False
    return df

In [201]:
def format_textcolumns(df,text_columns):
    for column in text_columns:
        df[column].replace({'&lt;':'<'},regex=True,inplace=True)
        df[column].replace({'\\n':' '},regex=True,inplace=True)
        df[column]=df['text'].str.replace('\s+', ' ', regex=True)
        df[column]=df[column].str.lower()
    return df

In [202]:
def handle_special_columns(df,list_columns,drop_columns,date_columns,notnan_columns,text_columns):
    df= explode_listcolumns(df,list_columns)
    df=df.drop(columns=drop_columns)
    df= format_datecolumns(df,date_columns)
    df= drop_row_notnan_columms(df,notnan_columns)
    df= format_textcolumns(df,text_columns)
    return df

In [203]:
#has to expanded to make it readeble by model
def preprocessing(df,topic_column,topics,topics_to_remove,list_columns,drop_columns,date_columns,notnan_columns,text_columns):
    df=categorize_topics(df,topic_column,topics,topics_to_remove)
    df = handle_special_columns(df,list_columns,drop_columns,date_columns,notnan_columns,text_columns)
    df['text_type'] = df['text_type'].replace({'"':''},regex=True)
    return df

In [204]:
df_processed = preprocessing(df,topic_column,topics,topics_to_remove,list_columns,drop_columns,date_columns,notnan_columns,text_columns)

C:\Users\theinrich\AppData\Local\Temp\ipykernel_20956\426969037.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column]=df[column].apply(lambda x : x[0])
C:\Users\theinrich\AppData\Local\Temp\ipykernel_20956\2598233951.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column]=series_to_list(df,column)
C:\Users\theinrich\AppData\Local\Temp\ipykernel_20956\2598233951.py:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace met

In [206]:
df_processed.head()

text text_type     topics  \
1   a survey of 19 provinces and seven cities show...      NORM       food   
6   tug crews in new south wales (nsw), victoria a...      NORM  transport   
9   western mining corp holdings ltd <wmng.s> (wmc...      NORM   resource   
10  sumitomo bank ltd <sumi.t> is certain to lose ...      NORM    finance   
11  mines and energy minister subroto confirmed in...      NORM   resource   

                      date                                              title  \
1  1987-04-08 01:19:17.290  a survey of 19 provinces and seven cities show...   
6  1987-04-08 02:42:14.860  tug crews in new south wales (nsw), victoria a...   
9  1987-04-08 03:21:39.740  western mining corp holdings ltd <wmng.s> (wmc...   
10 1987-04-08 03:29:34.510  sumitomo bank ltd <sumi.t> is certain to lose ...   
11 1987-04-08 03:45:20.530  mines and energy minister subroto confirmed in...   

    places_algeria  places_argentina  places_australia  places_austria  \
1              0.0               0.0               0.0             0.0   
6              0.0               0.0               1.0             0.0   
9              0.0               0.0               1.0             0.0   
10             0.0               0.0               0.0             0.0   
11             0.0               0.0               0.0             0.0   

    places_bahrain  ...  date_weekday_Thursday  date_weekday_Tuesday  \
1              0.0  ...                  False                 False   
6              0.0  ...                  False                 False   
9              0.0  ...                  False                 False   
10             0.0  ...                  False                 False   
11             0.0  ...                  False                 False   

    date_weekday_Wednesday  weekday_Monday  weekday_Tuesday  \
1                     True           False            False   
6                     True           False            False   
9                     True           False            False   
10                    True           False            False   
11                    True           False            False   

    weekday_Wednesday  weekday_Thursday  weekday_Friday  weekday_Saturday  \
1               False             False           False             False   
6               False             False           False             False   
9               False             False           False             False   
10              False             False           False             False   
11              False             False           False             False   

    weekday_Sunday  
1            False  
6            False  
9            False  
10           False  
11           False  

[5 rows x 106 columns]

In [207]:
df_processed['topics'].value_counts()


topics
personal_finance    1098
finance              979
resource             265
food                 127
transport             37
Name: count, dtype: int64